In [1]:
from typing import Any, Dict, Optional
import torch
import random
import os 
import argparse
from typing import (
    Any,
    Callable,
    Dict,
    Hashable,
    Iterable,
    List,
    Optional,
    Sequence,
    Tuple,
    Type,
    TypeVar,
    Union,
)
from botorch.utils.sampling import draw_sobol_samples

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from botorch.models.gp_regression import SingleTaskGP
from tqdm import tqdm
from torch import Tensor
from botorch.acquisition.active_learning import (
    MCSampler,
    qNegIntegratedPosteriorVariance,
)

from botorch.fit import fit_gpytorch_mll
from sklearn.model_selection import train_test_split
from botorch.models.gp_regression import SingleTaskGP

from sklearn.metrics import mean_absolute_error

import gc




from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
import warnings

import pickle

from botorch.exceptions.warnings import BotorchTensorDimensionWarning, InputDataWarning
warnings.filterwarnings(
            "ignore",
            message="Input data is not standardized.",
            category=InputDataWarning,
        )
import warnings
warnings.filterwarnings("ignore")



from botorch.models.fully_bayesian import SaasFullyBayesianSingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

from botorch.acquisition.active_learning import (
    MCSampler,
    qNegIntegratedPosteriorVariance,
)

from botorch.utils.transforms import normalize, standardize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# dtype = torch.double
dtype = torch.float32


# Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)


bounds = torch.tensor([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]], device=device, dtype=dtype)

mcp = draw_sobol_samples(bounds=bounds, n=1024, q=1, seed=42).squeeze(1)
mcp.to(device=device, dtype=dtype)
# bounds


seeds = np.load('seeds.npy')
xtest = np.load('xtest.npy')
ytest = np.load('ytest.npy')


with open('xcandidates_original.pkl', 'rb') as f:
    xcandidates_original = pickle.load(f)
    
with open('ycandidates_original.pkl', 'rb') as f:
    ycandidates_original = pickle.load(f)
    
xtest = torch.tensor(xtest, dtype=dtype,device=device)
ytest = torch.tensor(ytest, dtype=dtype,device=device)


In [5]:

def random_initial_data(x, y, initial_percent, seed=None):
    if seed is not None:
        np.random.seed(seed)
    # np.random.seed(seed)
    n = int(len(x)*initial_percent)
    idx = np.random.choice(len(x), n, replace=False).tolist()
    x_initial = [x[i] for i in idx]
    y_initial = [y[i] for i in idx]
    xcandidates = [x[i] for i in range(len(x)) if i not in idx]
    ycandidates = [y[i] for i in range(len(y)) if i not in idx]
    
    return x_initial, y_initial, xcandidates, ycandidates




rand_selection_mae = []
xmax_candidates = []
pred_mae = []
pred_y = []
pred_std = []
qnipv_runs =[]



def find_max_normalized_acqval(tensor_list, qNIVP):
    max_value = None
    max_index = -1
    acq_val_lst = []
    # torch.manual_seed(13)
    for i, tensor in enumerate(tensor_list):
        tensor_len = len(tensor)
        qNIVP_val = qNIVP(tensor)
        normalized_qNIVP_val = qNIVP_val / tensor_len
        acq_val_lst.append(normalized_qNIVP_val.item())
        if max_value is None or normalized_qNIVP_val > max_value:
            max_value = normalized_qNIVP_val
            max_index = i

    return max_value, max_index, acq_val_lst





for i in tqdm(seeds):
    print(f'running seed {i} of {len(seeds)}')
    torch.cuda.empty_cache()
    xcandidates = xcandidates_original.copy()
    ycandidates = ycandidates_original.copy()
    
    
    
    xinit, yinit, xcandidates, ycandidates = random_initial_data(xcandidates, ycandidates, 0.05, seed=i)
    
    
    xinit = torch.cat(xinit,dim=0).to(device)
    yinit = torch.cat(yinit,dim=0).to(device)

    
    gp = SingleTaskGP(xinit, yinit).to(device)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp).to(device)
    fit_gpytorch_mll(mll)
    
    with torch.no_grad():
        posterior = gp(xtest)
        ypred = posterior.mean.detach().cpu().numpy()
        # ystd = posterior.stddev.detach().cpu().numpy()
        del posterior
    
    # posterior = gp(xtest)
    # ypred = posterior.mean.detach().numpy()
    # ystd = posterior.stddev.detach().numpy()
    
    
    ymae = mean_absolute_error(ytest, ypred)
    
    pred_mae = []
    # pred_y.append(ypred)
    # pred_std.append(ystd)
    pred_mae.append(ymae)




    for inner_i in tqdm(range(100)):
        if not len(xcandidates):
            break
        
        qNIVP = qNegIntegratedPosteriorVariance(gp, mc_points= mcp)
        
        
        max_value, max_index, acq_val_lst = find_max_normalized_acqval(xcandidates, qNIVP)
        xmax_candidates.append(max_index)
        
        # print(f'pre-addtion of new ten',len(xinit))
        xinit= torch.cat((xinit, xcandidates[max_index]), 0).to(device)
        yinit = torch.cat((yinit, ycandidates[max_index]), 0).to(device)
        
        
        del xcandidates[max_index]
        del ycandidates[max_index]
        
        del gp, mll
        torch.cuda.empty_cache()
        gc.collect()
       
        
        gp = SingleTaskGP(xinit, yinit).to(device)
        # gp = SingleTaskGP(xinit, ytrain_,covar_module=rbf_kernel)
        mll = ExactMarginalLogLikelihood(gp.likelihood, gp).to(device)
        fit_gpytorch_mll(mll)
        
        with torch.no_grad():
            ypred = gp(xtest)
            ypred_mean = ypred.mean.detach().numpy()
            del ypred
            
            # pred_y.append(ypred_mean)

            ymae = mean_absolute_error(ytest, ypred_mean)
        # print('mean absolute error: ', ymae)
            pred_mae.append(ymae)
        # ystd = gp(xtest).stddev
        # ystd = ystd.detach().numpy()
        # pred_std.append(ystd)
    qnipv_runs.append(pred_mae)
    # np.save('qnipv_runs_copy.npy', np.array(qnipv_runs))
    np.save('qnipv_runs_seed.npy', np.array(pred_mae))

  0%|          | 0/25 [00:00<?, ?it/s]

running seed 25 of 25


  4%|▍         | 1/25 [19:48<7:55:17, 1188.22s/it]

running seed 1037 of 25


  8%|▊         | 2/25 [39:03<7:28:09, 1169.13s/it]

...

 92%|█████████▏| 23/25 [13:52:05<46:09, 1384.65s/it]  

running seed 1828 of 25


 96%|█████████▌| 24/25 [14:13:03<22:26, 1346.76s/it]

running seed 1698 of 25


100%|██████████| 25/25 [14:32:05<00:00, 2093.03s/it]


In [10]:
np.save('qnipv_runs_seed_true.npy', np.array(qnipv_runs))